In [272]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

- abir parquet

In [19]:
base = pd.read_parquet('../data/processed/df_final.parquet')

- Situação Vôo

In [20]:
colunas = ['SIGLA_ICAO_EMPRESA_AEREA', 'SITUACAO_VOO', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']
teste = pd.read_parquet('../data/processed/df_final.parquet', columns=colunas)

In [21]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   SIGLA_ICAO_EMPRESA_AEREA   717357 non-null  string  
 1   SITUACAO_VOO               717357 non-null  category
 2   MES_PARTIDA_PREVISTA       717357 non-null  int32   
 3   NOME_MES_PARTIDA_PREVISTA  717357 non-null  category
dtypes: category(2), int32(1), string(1)
memory usage: 9.6 MB


In [22]:
teste.head()

,SIGLA_ICAO_EMPRESA_AEREA,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA
0,AZU,REALIZADO,1,janeiro
1,AZU,REALIZADO,1,janeiro
2,AZU,REALIZADO,1,janeiro
3,AZU,REALIZADO,1,janeiro
4,AZU,REALIZADO,2,fevereiro


In [23]:
total = base.query(
    'SIGLA_ICAO_EMPRESA_AEREA == "TAM"  ' \
    ' and SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"'\
    ' and MES_PARTIDA_PREVISTA in [11,12] '
)
total

,NOME_AEROPORTO_ORIGEM,CIDADE_AEROPORTO_ORIGEM,UF_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NUMERO_VOO,CODIGO_TIPO_LINHA,SIGLA_ICAO_AEROPORTO_ORIGEM,SIGLA_ICAO_AEROPORTO_DESTINO,PARTIDA_PREVISTA,PARTIDA_REAL,...,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,DIA_DA_SEMANA_PARTIDA_REAL,INDICE_SEMANA_PARTIDA_REAL,DIA_DA_SEMANA_CHEGADA_REAL,DIA_DA_SEMANA_CHEGADA_PREVISTA,INDICE_SEMANA_CHEGADA_REAL,INDICE_SEMANA_CHEGADA_PREVISTA,FAIXA_ATRASO_PARTIDA,FAIXA_ATRASO_CHEGADA
537999,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3433,N,SBRP,SBGR,2022-11-01 09:30:00,2022-11-01 09:30:00,...,Terça-feira,1,Terça-feira,1,Terça-feira,Terça-feira,1,1,No horário,No horário
538000,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3433,N,SBRP,SBGR,2022-11-02 09:30:00,2022-11-02 09:30:00,...,Quarta-feira,2,Quarta-feira,2,Quarta-feira,Quarta-feira,2,2,No horário,No horário
538001,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3867,N,SBRP,SBGR,2022-11-02 14:20:00,2022-11-02 14:17:00,...,Quarta-feira,2,Quarta-feira,2,Quarta-feira,Quarta-feira,2,2,No horário,Atraso de até 1 hora
538002,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3433,N,SBRP,SBGR,2022-11-03 09:30:00,2022-11-03 09:25:00,...,Quinta-feira,3,Quinta-feira,3,Quinta-feira,Quinta-feira,3,3,No horário,No horário
538003,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3867,N,SBRP,SBGR,2022-11-03 14:20:00,2022-11-03 14:12:00,...,Quinta-feira,3,Quinta-feira,3,Quinta-feira,Quinta-feira,3,3,No horário,No horário
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538109,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3433,N,SBRP,SBGR,2022-12-29 09:30:00,2022-12-29 09:30:00,...,Quinta-feira,3,Quinta-feira,3,Quinta-feira,Quinta-feira,3,3,No horário,Atraso de até 30 minutos
538111,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3867,N,SBRP,SBGR,2022-12-29 14:20:00,2022-12-29 14:11:00,...,Quinta-feira,3,Quinta-feira,3,Quinta-feira,Quinta-feira,3,3,No horário,No horário
538112,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3433,N,SBRP,SBGR,2022-12-30 09:30:00,2022-12-30 09:29:00,...,Sexta-feira,4,Sexta-feira,4,Sexta-feira,Sexta-feira,4,4,No horário,No horário
538113,LEITE LOPES,RIBEIRÃO PRETO,SP,TAM,3867,N,SBRP,SBGR,2022-12-30 14:00:00,2022-12-30 13:50:00,...,Sexta-feira,4,Sexta-feira,4,Sexta-feira,Sexta-feira,4,4,No horário,No horário


In [24]:
total = base.query(
    'SIGLA_ICAO_EMPRESA_AEREA == "TAM"  ' \
    ' and SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"'\
    ' and MES_PARTIDA_PREVISTA in [11] '
)[
    [
        'NOME_MES_PARTIDA_PREVISTA',
        'SIGLA_ICAO_EMPRESA_AEREA', 
        'MES_PARTIDA_PREVISTA',
        'SITUACAO_VOO'
    ]
].value_counts()
# teste = base.query('SIGLA_ICAO_EMPRESA_AEREA == "TAP"')[['SIGLA_ICAO_EMPRESA_AEREA', 'SITUACAO_VOO']].value_counts( normalize=True) * 100
total


NOME_MES_PARTIDA_PREVISTA  SIGLA_ICAO_EMPRESA_AEREA  MES_PARTIDA_PREVISTA  SITUACAO_VOO
novembro                   TAM                       11                    REALIZADO       53
                                                                           CANCELADO        2
Name: count, dtype: int64

In [25]:
percentual = base.query(
    'SIGLA_ICAO_EMPRESA_AEREA == "TAM"  ' \
    ' and SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"' \
    ' and MES_PARTIDA_PREVISTA in [11]  '
    )[
        [
            'NOME_MES_PARTIDA_PREVISTA',
            'SIGLA_ICAO_EMPRESA_AEREA', 
            'MES_PARTIDA_PREVISTA',
            'SITUACAO_VOO'
        ]
    ].value_counts( normalize=True) * 100
percentual

NOME_MES_PARTIDA_PREVISTA  SIGLA_ICAO_EMPRESA_AEREA  MES_PARTIDA_PREVISTA  SITUACAO_VOO
novembro                   TAM                       11                    REALIZADO       96.363636
                                                                           CANCELADO        3.636364
Name: proportion, dtype: float64

In [26]:
df_resultado = pd.concat([total, percentual], axis=1).reset_index()
df_resultado

,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_EMPRESA_AEREA,MES_PARTIDA_PREVISTA,SITUACAO_VOO,count,proportion
0,novembro,TAM,11,REALIZADO,53,96.363636
1,novembro,TAM,11,CANCELADO,2,3.636364


In [27]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   NOME_MES_PARTIDA_PREVISTA  2 non-null      category
 1   SIGLA_ICAO_EMPRESA_AEREA   2 non-null      string  
 2   MES_PARTIDA_PREVISTA       2 non-null      int32   
 3   SITUACAO_VOO               2 non-null      category
 4   count                      2 non-null      int64   
 5   proportion                 2 non-null      float64 
dtypes: category(2), float64(1), int32(1), int64(1), string(1)
memory usage: 716.0 bytes


In [28]:
df_resultado['count'] = df_resultado['count'].astype('int32')
df_resultado['proportion'] = df_resultado['proportion'].astype('float32')

In [31]:
df_resultado


,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_EMPRESA_AEREA,MES_PARTIDA_PREVISTA,SITUACAO_VOO,count,proportion
0,novembro,TAM,11,REALIZADO,53,96.363640
1,novembro,TAM,11,CANCELADO,2,3.636364


In [30]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   NOME_MES_PARTIDA_PREVISTA  2 non-null      category
 1   SIGLA_ICAO_EMPRESA_AEREA   2 non-null      string  
 2   MES_PARTIDA_PREVISTA       2 non-null      int32   
 3   SITUACAO_VOO               2 non-null      category
 4   count                      2 non-null      int32   
 5   proportion                 2 non-null      float32 
dtypes: category(2), float32(1), int32(2), string(1)
memory usage: 700.0 bytes


- Desempenho da Situação do Voo

In [32]:
colunas = ['SIGLA_ICAO_EMPRESA_AEREA', 'SITUACAO_VOO', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA', 'SIGLA_ICAO_AEROPORTO_ORIGEM']
teste = pd.read_parquet('../data/processed/df_final.parquet', columns=colunas)

In [33]:
teste = teste.query('SIGLA_ICAO_EMPRESA_AEREA == "TAM"  ' \
            ' and SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"'
            ' and SITUACAO_VOO == "REALIZADO" ' 
            )

teste.head()

,SIGLA_ICAO_EMPRESA_AEREA,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM
488380,TAM,REALIZADO,1,janeiro,SBRP
488381,TAM,REALIZADO,1,janeiro,SBRP
537779,TAM,REALIZADO,1,janeiro,SBRP
537780,TAM,REALIZADO,1,janeiro,SBRP
537781,TAM,REALIZADO,1,janeiro,SBRP


In [34]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345 entries, 488380 to 680870
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   SIGLA_ICAO_EMPRESA_AEREA     345 non-null    string  
 1   SITUACAO_VOO                 345 non-null    category
 2   MES_PARTIDA_PREVISTA         345 non-null    int32   
 3   NOME_MES_PARTIDA_PREVISTA    345 non-null    category
 4   SIGLA_ICAO_AEROPORTO_ORIGEM  345 non-null    string  
dtypes: category(2), int32(1), string(2)
memory usage: 10.6 KB


In [35]:
teste['SITUACAO_VOO'] = teste['SITUACAO_VOO'].astype('string')
teste['NOME_MES_PARTIDA_PREVISTA'] = teste['NOME_MES_PARTIDA_PREVISTA'].astype('string')
teste.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345 entries, 488380 to 680870
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   SIGLA_ICAO_EMPRESA_AEREA     345 non-null    string
 1   SITUACAO_VOO                 345 non-null    string
 2   MES_PARTIDA_PREVISTA         345 non-null    int32 
 3   NOME_MES_PARTIDA_PREVISTA    345 non-null    string
 4   SIGLA_ICAO_AEROPORTO_ORIGEM  345 non-null    string
dtypes: int32(1), string(4)
memory usage: 14.8 KB


In [36]:
teste_gb = teste.groupby(['MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA', 'SITUACAO_VOO'], observed=False).agg(
    TOTAL_SITUACAO=('SITUACAO_VOO', 'count')
)
teste_gb.sort_values(by=['MES_PARTIDA_PREVISTA'], inplace=True)
teste_gb.reset_index(inplace=True)
teste_gb

,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SITUACAO_VOO,TOTAL_SITUACAO
0,1,janeiro,REALIZADO,41
1,2,fevereiro,REALIZADO,20
2,3,março,REALIZADO,23
3,4,abril,REALIZADO,21
4,5,maio,REALIZADO,20
5,6,junho,REALIZADO,22
6,7,julho,REALIZADO,21
7,8,agosto,REALIZADO,23
8,9,setembro,REALIZADO,23
9,10,outubro,REALIZADO,24


In [37]:
fig = px.bar(
    teste_gb, 
    x='NOME_MES_PARTIDA_PREVISTA', 
    y='TOTAL_SITUACAO', 
    text_auto=True,
)
fig.update_traces(
    textposition='outside'
)
fig.show()

- Total de Vôos por aeroporto

In [38]:
colunas = ['NOME_AEROPORTO_ORIGEM', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA', 'SIGLA_ICAO_AEROPORTO_ORIGEM']
teste = pd.read_parquet('../data/processed/df_final.parquet', columns=colunas)

In [39]:
teste.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"')

,NOME_AEROPORTO_ORIGEM,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM
1929,LEITE LOPES,1,janeiro,SBRP
1930,LEITE LOPES,1,janeiro,SBRP
1931,LEITE LOPES,1,janeiro,SBRP
1932,LEITE LOPES,1,janeiro,SBRP
1933,LEITE LOPES,1,janeiro,SBRP
...,...,...,...,...
707275,LEITE LOPES,1,janeiro,SBRP
707479,LEITE LOPES,1,janeiro,SBRP
707480,LEITE LOPES,1,janeiro,SBRP
707481,LEITE LOPES,1,janeiro,SBRP


In [40]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 4 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   NOME_AEROPORTO_ORIGEM        717357 non-null  string  
 1   MES_PARTIDA_PREVISTA         717357 non-null  int32   
 2   NOME_MES_PARTIDA_PREVISTA    717357 non-null  category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM  717357 non-null  string  
dtypes: category(1), int32(1), string(2)
memory usage: 14.4 MB


In [41]:
teste['NOME_MES_PARTIDA_PREVISTA'] = teste['NOME_MES_PARTIDA_PREVISTA'].astype('string')

In [42]:
total_voos =teste.groupby(['MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('NOME_AEROPORTO_ORIGEM', 'count')
)
total_voos.reset_index(inplace=True)
total_voos['TOTAL_VOOS'] = total_voos['TOTAL_VOOS'].astype('int32')


In [240]:
total_voos

,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,TOTAL_VOOS
0,1,janeiro,78523
1,2,fevereiro,46597
2,3,março,54542
3,4,abril,52739
4,5,maio,58631
5,6,junho,55320
6,7,julho,63666
7,8,agosto,60677
8,9,setembro,58683
9,10,outubro,61976


In [43]:
total_voos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   MES_PARTIDA_PREVISTA       12 non-null     int32 
 1   NOME_MES_PARTIDA_PREVISTA  12 non-null     string
 2   TOTAL_VOOS                 12 non-null     int32 
dtypes: int32(2), string(1)
memory usage: 320.0 bytes


In [44]:
fig = px.bar(
    total_voos, 
    x='NOME_MES_PARTIDA_PREVISTA', 
    y='TOTAL_VOOS', 
    text_auto='0',
)
fig.update_traces(
    textposition='outside'
)
fig.show()

- Total de Vôos por compania e aeroporto por mês

In [45]:
base.columns

Index(['NOME_AEROPORTO_ORIGEM', 'CIDADE_AEROPORTO_ORIGEM',
       'UF_AEROPORTO_ORIGEM', 'SIGLA_ICAO_EMPRESA_AEREA', 'NUMERO_VOO',
       'CODIGO_TIPO_LINHA', 'SIGLA_ICAO_AEROPORTO_ORIGEM',
       'SIGLA_ICAO_AEROPORTO_DESTINO', 'PARTIDA_PREVISTA', 'PARTIDA_REAL',
       'CHEGADA_PREVISTA', 'CHEGADA_REAL', 'SITUACAO_VOO', 'SIGLA_OACI',
       'NOME_AEROPORTO_DESTINO', 'CIDADE_AEROPORTO_DESTINO',
       'UF_AEROPORTO_DESTINO', 'NOME_EMPRESA', 'MES_PARTIDA_PREVISTA',
       'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_REAL',
       'NOME_MES_PARTIDA_REAL', 'MES_CHEGADA_PREVISTA',
       'NOME_MES_CHEGADA_PREVISTA', 'MES_CHEGADA_REAL',
       'NOME_MES_CHEGADA_REAL', 'MINUTOS_ATRASADO_PARTIDA',
       'MINUTOS_ATRASADO_CHEGADA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA',
       'INDICE_SEMANA_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_REAL',
       'INDICE_SEMANA_PARTIDA_REAL', 'DIA_DA_SEMANA_CHEGADA_REAL',
       'DIA_DA_SEMANA_CHEGADA_PREVISTA', 'INDICE_SEMANA_CHEGADA_REAL',
       'INDICE_SEMANA_CH

In [46]:
colunas = [
        
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'MES_PARTIDA_PREVISTA',
        'NOME_MES_PARTIDA_PREVISTA',
        'NUMERO_VOO' ,
        'SIGLA_ICAO_EMPRESA_AEREA',
        'NOME_EMPRESA' 
]


teste = pd.read_parquet('../data/processed/df_final.parquet', columns=colunas)
teste = teste.query(
    ' SIGLA_ICAO_EMPRESA_AEREA == "TAM"  ' \
    ' and SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"'\
)

In [47]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348 entries, 488380 to 680870
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   SIGLA_ICAO_AEROPORTO_ORIGEM  348 non-null    string  
 1   MES_PARTIDA_PREVISTA         348 non-null    int32   
 2   NOME_MES_PARTIDA_PREVISTA    348 non-null    category
 3   NUMERO_VOO                   348 non-null    string  
 4   SIGLA_ICAO_EMPRESA_AEREA     348 non-null    string  
 5   NOME_EMPRESA                 348 non-null    string  
dtypes: category(1), int32(1), string(4)
memory usage: 15.7 KB


In [48]:
teste['NOME_MES_PARTIDA_PREVISTA'] = teste['NOME_MES_PARTIDA_PREVISTA'].astype('string')

In [49]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348 entries, 488380 to 680870
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   SIGLA_ICAO_AEROPORTO_ORIGEM  348 non-null    string
 1   MES_PARTIDA_PREVISTA         348 non-null    int32 
 2   NOME_MES_PARTIDA_PREVISTA    348 non-null    string
 3   NUMERO_VOO                   348 non-null    string
 4   SIGLA_ICAO_EMPRESA_AEREA     348 non-null    string
 5   NOME_EMPRESA                 348 non-null    string
dtypes: int32(1), string(5)
memory usage: 17.7 KB


In [50]:
teste.head()

,SIGLA_ICAO_AEROPORTO_ORIGEM,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,NUMERO_VOO,SIGLA_ICAO_EMPRESA_AEREA,NOME_EMPRESA
488380,SBRP,1,janeiro,3867,TAM,TAM
488381,SBRP,1,janeiro,3867,TAM,TAM
537779,SBRP,1,janeiro,3992,TAM,TAM
537780,SBRP,1,janeiro,3992,TAM,TAM
537781,SBRP,1,janeiro,3992,TAM,TAM


In [51]:
teste_gb = teste.groupby([ 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('SIGLA_ICAO_EMPRESA_AEREA', 'count')
).reset_index()

In [52]:
teste_gb

,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,TOTAL_VOOS
0,1,janeiro,41
1,2,fevereiro,20
2,3,março,23
3,4,abril,21
4,5,maio,20
5,6,junho,22
6,7,julho,21
7,8,agosto,23
8,9,setembro,23
9,10,outubro,24


In [53]:
fig = px.bar(
    teste_gb, 
    x='NOME_MES_PARTIDA_PREVISTA', 
    y='TOTAL_VOOS', 
    text_auto='0',
)
fig.update_traces(
    textposition='outside'
)
fig.show()

- Analise da variação percentual da SITUAÇÂO voo Filtrando por Aerporto origem por mês (deixe filtro facultativo por empresa)

In [54]:
base.columns

Index(['NOME_AEROPORTO_ORIGEM', 'CIDADE_AEROPORTO_ORIGEM',
       'UF_AEROPORTO_ORIGEM', 'SIGLA_ICAO_EMPRESA_AEREA', 'NUMERO_VOO',
       'CODIGO_TIPO_LINHA', 'SIGLA_ICAO_AEROPORTO_ORIGEM',
       'SIGLA_ICAO_AEROPORTO_DESTINO', 'PARTIDA_PREVISTA', 'PARTIDA_REAL',
       'CHEGADA_PREVISTA', 'CHEGADA_REAL', 'SITUACAO_VOO', 'SIGLA_OACI',
       'NOME_AEROPORTO_DESTINO', 'CIDADE_AEROPORTO_DESTINO',
       'UF_AEROPORTO_DESTINO', 'NOME_EMPRESA', 'MES_PARTIDA_PREVISTA',
       'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_REAL',
       'NOME_MES_PARTIDA_REAL', 'MES_CHEGADA_PREVISTA',
       'NOME_MES_CHEGADA_PREVISTA', 'MES_CHEGADA_REAL',
       'NOME_MES_CHEGADA_REAL', 'MINUTOS_ATRASADO_PARTIDA',
       'MINUTOS_ATRASADO_CHEGADA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA',
       'INDICE_SEMANA_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_REAL',
       'INDICE_SEMANA_PARTIDA_REAL', 'DIA_DA_SEMANA_CHEGADA_REAL',
       'DIA_DA_SEMANA_CHEGADA_PREVISTA', 'INDICE_SEMANA_CHEGADA_REAL',
       'INDICE_SEMANA_CH

In [65]:
colunas = [
    'SITUACAO_VOO',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
]
teste = pd.read_parquet('../data/processed/df_final.parquet', columns=colunas)

In [66]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   SITUACAO_VOO                 717357 non-null  category
 1   MES_PARTIDA_PREVISTA         717357 non-null  int32   
 2   NOME_MES_PARTIDA_PREVISTA    717357 non-null  category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM  717357 non-null  string  
 4   NOME_AEROPORTO_ORIGEM        717357 non-null  string  
dtypes: category(2), int32(1), string(2)
memory usage: 15.1 MB


In [67]:
teste.head()

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM
0,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA
1,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA
2,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA
3,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA
4,REALIZADO,2,fevereiro,SBAE,BAURU/AREALVA


In [68]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   SITUACAO_VOO                 717357 non-null  category
 1   MES_PARTIDA_PREVISTA         717357 non-null  int32   
 2   NOME_MES_PARTIDA_PREVISTA    717357 non-null  category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM  717357 non-null  string  
 4   NOME_AEROPORTO_ORIGEM        717357 non-null  string  
dtypes: category(2), int32(1), string(2)
memory usage: 15.1 MB


In [69]:
teste[['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA']] = teste[['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA']].astype('string')

In [70]:
teste = teste.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"')
teste

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM
1929,REALIZADO,1,janeiro,SBRP,LEITE LOPES
1930,REALIZADO,1,janeiro,SBRP,LEITE LOPES
1931,REALIZADO,1,janeiro,SBRP,LEITE LOPES
1932,REALIZADO,1,janeiro,SBRP,LEITE LOPES
1933,REALIZADO,1,janeiro,SBRP,LEITE LOPES
...,...,...,...,...,...
707275,REALIZADO,1,janeiro,SBRP,LEITE LOPES
707479,REALIZADO,1,janeiro,SBRP,LEITE LOPES
707480,REALIZADO,1,janeiro,SBRP,LEITE LOPES
707481,REALIZADO,1,janeiro,SBRP,LEITE LOPES


In [71]:
teste_gb =teste.groupby(['NOME_MES_PARTIDA_PREVISTA', 'SITUACAO_VOO', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('SITUACAO_VOO', 'count')
)

teste_gb.reset_index(inplace=True)
teste_gb.sort_values(by=['MES_PARTIDA_PREVISTA'], inplace=True)
teste_gb['TOTAL_VOOS'] = teste_gb['TOTAL_VOOS'].astype('int32')

In [72]:
teste_gb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 8 to 4
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   NOME_MES_PARTIDA_PREVISTA  24 non-null     string
 1   SITUACAO_VOO               24 non-null     string
 2   MES_PARTIDA_PREVISTA       24 non-null     int32 
 3   TOTAL_VOOS                 24 non-null     int32 
dtypes: int32(2), string(2)
memory usage: 768.0 bytes


In [73]:
teste_gb

,NOME_MES_PARTIDA_PREVISTA,SITUACAO_VOO,MES_PARTIDA_PREVISTA,TOTAL_VOOS
8,janeiro,CANCELADO,1,21
9,janeiro,REALIZADO,1,584
6,fevereiro,CANCELADO,2,18
7,fevereiro,REALIZADO,2,249
17,março,REALIZADO,3,345
16,março,CANCELADO,3,16
0,abril,CANCELADO,4,3
1,abril,REALIZADO,4,381
15,maio,REALIZADO,5,433
14,maio,CANCELADO,5,14


In [77]:
fig = px.bar(
    teste_gb, 
    x='NOME_MES_PARTIDA_PREVISTA', 
    y='TOTAL_VOOS', 
    color='SITUACAO_VOO',
    barmode='group',
    text_auto=True
)
fig.update_traces(
    textposition='outside'
)
fig.show()

- Analise da variação percentual da SITUAÇÂO voo Filtrando por Aerporto origem por mês compare com o mês anterior (Talvez com vísual cartões) deixe filtro facultativo por empresa)

In [79]:
base.columns

Index(['NOME_AEROPORTO_ORIGEM', 'CIDADE_AEROPORTO_ORIGEM',
       'UF_AEROPORTO_ORIGEM', 'SIGLA_ICAO_EMPRESA_AEREA', 'NUMERO_VOO',
       'CODIGO_TIPO_LINHA', 'SIGLA_ICAO_AEROPORTO_ORIGEM',
       'SIGLA_ICAO_AEROPORTO_DESTINO', 'PARTIDA_PREVISTA', 'PARTIDA_REAL',
       'CHEGADA_PREVISTA', 'CHEGADA_REAL', 'SITUACAO_VOO', 'SIGLA_OACI',
       'NOME_AEROPORTO_DESTINO', 'CIDADE_AEROPORTO_DESTINO',
       'UF_AEROPORTO_DESTINO', 'NOME_EMPRESA', 'MES_PARTIDA_PREVISTA',
       'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_REAL',
       'NOME_MES_PARTIDA_REAL', 'MES_CHEGADA_PREVISTA',
       'NOME_MES_CHEGADA_PREVISTA', 'MES_CHEGADA_REAL',
       'NOME_MES_CHEGADA_REAL', 'MINUTOS_ATRASADO_PARTIDA',
       'MINUTOS_ATRASADO_CHEGADA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA',
       'INDICE_SEMANA_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_REAL',
       'INDICE_SEMANA_PARTIDA_REAL', 'DIA_DA_SEMANA_CHEGADA_REAL',
       'DIA_DA_SEMANA_CHEGADA_PREVISTA', 'INDICE_SEMANA_CHEGADA_REAL',
       'INDICE_SEMANA_CH

In [250]:
colunas = [
    'SITUACAO_VOO',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'NOME_EMPRESA'

]
teste = pd.read_parquet('../data/processed/df_final.parquet', columns=colunas)

In [251]:
teste.head()

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NOME_EMPRESA
0,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA,AZU,AZUL
1,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA,AZU,AZUL
2,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA,AZU,AZUL
3,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA,AZU,AZUL
4,REALIZADO,2,fevereiro,SBAE,BAURU/AREALVA,AZU,AZUL


In [252]:
teste = teste.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"  and SIGLA_ICAO_EMPRESA_AEREA =="AZU" ')
teste.head()

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NOME_EMPRESA
1929,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1930,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1931,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1932,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1933,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL


In [253]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2361 entries, 1929 to 258326
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   SITUACAO_VOO                 2361 non-null   category
 1   MES_PARTIDA_PREVISTA         2361 non-null   int32   
 2   NOME_MES_PARTIDA_PREVISTA    2361 non-null   category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM  2361 non-null   string  
 4   NOME_AEROPORTO_ORIGEM        2361 non-null   string  
 5   SIGLA_ICAO_EMPRESA_AEREA     2361 non-null   string  
 6   NOME_EMPRESA                 2361 non-null   string  
dtypes: category(2), int32(1), string(4)
memory usage: 106.6 KB


In [254]:
teste[['SITUACAO_VOO','NOME_MES_PARTIDA_PREVISTA' ]] = teste[['SITUACAO_VOO','NOME_MES_PARTIDA_PREVISTA' ]].astype('string')

In [255]:
df_gb = teste.groupby(['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_SITUACAO=('SITUACAO_VOO', 'count')
).sort_values(by=['SITUACAO_VOO','MES_PARTIDA_PREVISTA']).reset_index()
df_gb

,SITUACAO_VOO,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_SITUACAO
0,CANCELADO,janeiro,1,17
1,CANCELADO,fevereiro,2,2
2,CANCELADO,março,3,2
3,CANCELADO,maio,5,9
4,CANCELADO,junho,6,13
5,CANCELADO,julho,7,4
6,CANCELADO,agosto,8,3
7,CANCELADO,setembro,9,11
8,CANCELADO,outubro,10,15
9,CANCELADO,novembro,11,10


In [256]:
df_gb['TOTAL_SITUACAO_MES_ANTERIOR'] = df_gb.groupby('SITUACAO_VOO')['TOTAL_SITUACAO'].shift(1)
df_gb.fillna(0, axis=1, inplace=True)
df_gb

,SITUACAO_VOO,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR
0,CANCELADO,janeiro,1,17,0
1,CANCELADO,fevereiro,2,2,17.0
2,CANCELADO,março,3,2,2.0
3,CANCELADO,maio,5,9,2.0
4,CANCELADO,junho,6,13,9.0
5,CANCELADO,julho,7,4,13.0
6,CANCELADO,agosto,8,3,4.0
7,CANCELADO,setembro,9,11,3.0
8,CANCELADO,outubro,10,15,11.0
9,CANCELADO,novembro,11,10,15.0


In [257]:
def calcular_variacao(linha):
    try:
        total_atual = linha['TOTAL_SITUACAO']
        total_mes_anterior = linha['TOTAL_SITUACAO_MES_ANTERIOR']
        variacao = ((total_atual - total_mes_anterior) / total_mes_anterior) * 100
    except ZeroDivisionError:
        variacao = 0
    return round(variacao, 2)


In [258]:
df_gb['PERCENTUAL_VARIACAO'] = df_gb.apply(calcular_variacao, axis=1)
df_gb

,SITUACAO_VOO,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR,PERCENTUAL_VARIACAO
0,CANCELADO,janeiro,1,17,0,0.00
1,CANCELADO,fevereiro,2,2,17.0,-88.24
2,CANCELADO,março,3,2,2.0,0.00
3,CANCELADO,maio,5,9,2.0,350.00
4,CANCELADO,junho,6,13,9.0,44.44
5,CANCELADO,julho,7,4,13.0,-69.23
6,CANCELADO,agosto,8,3,4.0,-25.00
7,CANCELADO,setembro,9,11,3.0,266.67
8,CANCELADO,outubro,10,15,11.0,36.36
9,CANCELADO,novembro,11,10,15.0,-33.33


In [270]:
df_status_cancelado = df_gb.query('SITUACAO_VOO == "CANCELADO" and MES_PARTIDA_PREVISTA == 2')[['TOTAL_SITUACAO', 'TOTAL_SITUACAO_MES_ANTERIOR', 'PERCENTUAL_VARIACAO']]
df_status_cancelado

,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR,PERCENTUAL_VARIACAO
1,2,17.0,-88.24


In [271]:
df_status_realizado = df_gb.query('SITUACAO_VOO == "REALIZADO" and MES_PARTIDA_PREVISTA == 2')[['TOTAL_SITUACAO', 'TOTAL_SITUACAO_MES_ANTERIOR', 'PERCENTUAL_VARIACAO']]
df_status_realizado

,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR,PERCENTUAL_VARIACAO
12,144,240.0,-40.0


In [284]:
fig = go.Figure(
    go.Indicator(
        mode='number+delta',
        value=df_status_realizado['TOTAL_SITUACAO'].values[0],
        delta={
            'position': 'top', 
            'reference': df_status_realizado['TOTAL_SITUACAO_MES_ANTERIOR'].values[0], 
            'relative': True,
        },
        title={
            'text': 'Total Situação Vôos Realizados'
        }
    )
)
fig.show()

- Composição das situação do voo filtre por aeroporto em empresa

In [310]:
colunas = [
    'SITUACAO_VOO',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'NOME_EMPRESA'

]
teste = pd.read_parquet('../data/processed/df_final.parquet', columns=colunas)

In [311]:
teste = teste.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"  and SIGLA_ICAO_EMPRESA_AEREA =="AZU"') #  and SITUACAO_VOO=="REALIZADO"
teste.head()

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NOME_EMPRESA
1929,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1930,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1931,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1932,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1933,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL


In [312]:
teste[['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA']] = teste[['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA']].astype('string')

In [313]:
teste_gb = teste.groupby(['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('NOME_MES_PARTIDA_PREVISTA', 'count')
).reset_index().sort_values(by=['MES_PARTIDA_PREVISTA'])
teste_gb

,SITUACAO_VOO,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_VOOS
3,CANCELADO,janeiro,1,17
15,REALIZADO,janeiro,1,240
2,CANCELADO,fevereiro,2,2
14,REALIZADO,fevereiro,2,144
19,REALIZADO,março,3,171
7,CANCELADO,março,3,2
11,REALIZADO,abril,4,194
6,CANCELADO,maio,5,9
18,REALIZADO,maio,5,212
5,CANCELADO,junho,6,13


In [318]:
teste_total = teste_gb.groupby([ 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOO_MES=('TOTAL_VOOS', 'sum')
).reset_index().sort_values(by=['MES_PARTIDA_PREVISTA'])
teste_total

,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_VOO_MES
4,janeiro,1,257
3,fevereiro,2,146
8,março,3,173
0,abril,4,194
7,maio,5,221
6,junho,6,203
5,julho,7,223
1,agosto,8,198
11,setembro,9,196
10,outubro,10,195


In [323]:

def obter_variacao(linha):
    try:
        total_voo_mes = linha['TOTAL_VOO_MES']
        total_voo = linha['TOTAL_VOOS']
        variacao = (total_voo * 100) / total_voo_mes
    except ZeroDivisionError:
        variacao = 0
    return round(variacao, 2)


In [322]:
teste_completo = teste_total.merge(teste_gb, how='inner', on='NOME_MES_PARTIDA_PREVISTA')
teste_completo[['NOME_MES_PARTIDA_PREVISTA', 'SITUACAO_VOO', 'TOTAL_VOOS', 'TOTAL_VOO_MES']]

,NOME_MES_PARTIDA_PREVISTA,SITUACAO_VOO,TOTAL_VOOS,TOTAL_VOO_MES
0,janeiro,CANCELADO,17,257
1,janeiro,REALIZADO,240,257
2,fevereiro,CANCELADO,2,146
3,fevereiro,REALIZADO,144,146
4,março,REALIZADO,171,173
5,março,CANCELADO,2,173
6,abril,REALIZADO,194,194
7,maio,CANCELADO,9,221
8,maio,REALIZADO,212,221
9,junho,CANCELADO,13,203


In [324]:
teste_completo['VARIACAO_MENSAL'] = teste_completo.apply(obter_variacao, axis=1)
teste_completo

,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA_x,TOTAL_VOO_MES,SITUACAO_VOO,MES_PARTIDA_PREVISTA_y,TOTAL_VOOS,VARIACAO_MENSAL
0,janeiro,1,257,CANCELADO,1,17,6.61
1,janeiro,1,257,REALIZADO,1,240,93.39
2,fevereiro,2,146,CANCELADO,2,2,1.37
3,fevereiro,2,146,REALIZADO,2,144,98.63
4,março,3,173,REALIZADO,3,171,98.84
5,março,3,173,CANCELADO,3,2,1.16
6,abril,4,194,REALIZADO,4,194,100.00
7,maio,5,221,CANCELADO,5,9,4.07
8,maio,5,221,REALIZADO,5,212,95.93
9,junho,6,203,CANCELADO,6,13,6.40


- Compare total de vôos realizado no fim de semana em relação por dia útil por mês

<p> - Analise a FAIXA_ATRASO_PARTIDA por aeroporto e empresa por dia de semana e por mês </p>
<p> O volume de atraso no fim de semana, é maior ou menor em relação ao dia atual</p>
<p> Pode ser análisado a incidencia por aeroporto também</p>
<p>Detalhe por semana também</p>
<p>Eu tenho 3000 vôos que partiram no horário 50%(1500) da empresa A e 50% da empresa B</p>


<p> - Analise a FAIXA_ATRASO_CHEGADA por aeroporto e empresa por dia de semana e por mês </p>
<p> O volume de atraso no fim de semana, é maior ou menor em relação ao dia atual</p>
<p> Pode ser análisado a incidencia por aeroporto também</p>
<p>Detalhe por semana também</p>
<p>Eu tenho 3000 vôos que partiram no horário 50%(1500) da empresa A e 50% da empresa B</p>


<p> Ranking por FAIXA_ATRASO_CHEGADA selecionando aeroporto  por mês e empresa</p>

<p> Ranking por FAIXA_ATRASO_SAIDA selecionando aeroporto em empresa por mês</p>

<p> Quantifique a porcentagem por FAIXA_ATRASO_CHEGADA selecionando aeroportoa por mês</p>

<p> Quantifique a porcentagem por FAIXA_ATRASO_PARTIFA selecionando aeroporto  por mês</p>